In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client
import datetime
import pandas as pd
import pickle
import numpy as np

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

# Проверка нахождения клиента в черном списке с использованием Pandas Data Frame
def black_list_check(full_name, birth):
    # Разбиваем строку с ФИО на отдельные части (фамилия, имя, отчество)
    parts = full_name.split()
    
    if len(parts) != 3:
        return "Введите корректное ФИО клиента"
    
    # Извлекаем фамилию, имя и отчество из строки
    surname, name, patronym = parts
    
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    
    # Поиск совпадений по ФИО и дате рождения
    exist = any((frame['Surname'] == surname) & (frame['Name'] == name) & (frame['Patronym'] == patronym) & (frame['Birth'] == birth))
    
    if exist:
        return f"{surname} {name} {patronym} with the date of birth: {birth}: bad_boy"
    else:
        return f"{surname} {name} {patronym} with the date of birth: {birth}: good_boy"

server.register_function(black_list_check, 'black_list_check')


# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary,'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
# или монохромное изображение размерностью (M, N, 1)
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3 and img_arr.shape[2] == 3:
        # Если изображение имеет трехканальный формат RGB
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
    elif len(img_arr.shape) == 2:
        # Если изображение монохромное (градации серого)
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion,'send_back_inversion')

# Бинаризация изображения по заданному порогу
def threshold_image(bin_data, threshold):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    binary_img_arr = np.zeros_like(img_arr, dtype=np.uint8)
    
    if len(img_arr.shape) == 3:
        # Если изображение имеет трехканальный формат RGB
        for i in range(height):
            for j in range(width):
                # Вычисление яркости пикселя для RGB
                brightness = int(0.2126 * img_arr[i][j][0] + 0.7152 * img_arr[i][j][1] + 0.0722 * img_arr[i][j][2])
                if brightness > threshold:
                    binary_img_arr[i][j] = [255, 255, 255]
                else:
                    binary_img_arr[i][j] = [0, 0, 0]
    elif len(img_arr.shape) == 2:
        # Если изображение монохромное (градации серого)
        for i in range(height):
            for j in range(width):
                if img_arr[i][j] > threshold:
                    binary_img_arr[i][j] = 255
                else:
                    binary_img_arr[i][j] = 0

    pimg = pickle.dumps(binary_img_arr)        
    return xmlrpc.client.Binary(pimg)
# Регистрация функции бинаризации
server.register_function(threshold_image,'threshold_image')

# Разворот изображения относительно вертикали
def flip_image_vertical(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    
    # Создание нового массива для развернутого изображения
    flipped_img_arr = np.zeros_like(img_arr, dtype=np.uint8)
    
    if len(img_arr.shape) == 3:
        # Если изображение имеет трехканальный формат RGB
        for i in range(height):
            flipped_img_arr[height - i - 1] = img_arr[i]
    elif len(img_arr.shape) == 2:
        # Если изображение монохромное (градации серого)
        for i in range(height):
            flipped_img_arr[height - i - 1] = img_arr[i]
    
    pimg = pickle.dumps(flipped_img_arr)        
    return xmlrpc.client.Binary(pimg)
# Регистрация функции разворота
server.register_function(flip_image_vertical,'flip_image_vertical')

print("Listening on port 8008...")
server.serve_forever()

Listening on port 8008...


127.0.0.1 - - [24/Oct/2023 17:11:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:11:07] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:11:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:00] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:20] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:25] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:32] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2023 17:12:36] "POST /RPC2 HTTP/1.1" 200 -
